In [16]:
from z3 import *
from utils import *
from math import ceil

### Instances

In [1]:
import os

def read_dat_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # Read m and n
    m = int(lines[0].strip())
    n = int(lines[1].strip())
    
    # Read l vector
    l = list(map(int, lines[2].strip().split()))
    
    # Read s vector
    s = list(map(int, lines[3].strip().split()))
    
    # Read D matrix
    D = []
    for line in lines[4:]:
        D.append(list(map(int, line.strip().split())))
    
    return {
        'm': m,
        'n': n,
        'l': l,
        's': s,
        'D': D
    }

def read_all_dat_files(directory):
    instances = []
    for filename in os.listdir(directory):
        if filename.endswith('.dat'):
            file_path = os.path.join(directory, filename)
            instance = read_dat_file(file_path)
            instances.append(instance)
    return instances

# Directory containing .dat files
directory = 'instances'

# Read all .dat files and populate instances
instances = read_all_dat_files(directory)


In [21]:
for i in instances:
    max_element = [max(max(row) for row in i['D'])]

max_d = max(max_element)
print(max_d)

339


### MCP

In [ ]:

def MCP(instance, timeout = 5000):

    m = instance['m']
    n = instance['n']
    l = instance['l']
    s = instance['s']
    D = instance['D']

    path = [[[Bool(f"p_{i}_{j}_{k}") for k in range(n)] for j in range(n)] for i in range(m)]

    # considerando
    max_cost = ceil(n/m)*max_d
    
    s = Solver()

    s.set(timeout=timeout) # priority='pareto
    
    for i in range(n):
        for t in range(n): 
            s.add(exactly_one_seq([path[c][t][i] for c in range(m)]))

    for c in range(m):
        for t in range(n): 
            s.add(exactly_one_seq([path[c][t][i] for i in range(n)]))

    # each item is picked by only a courier
    for c in range(m):
        for i in range(n): 
            s.add(exactly_one_seq([path[c][t][i] for t in range(n)]))

    # Maximum load constraint
    for i in range(m):
        weigth_set = []
        for j in range(n):
            for k in range(n):
                weigth_set.extend([path[i][j][k]] * s[j])
        s.add(at_most_k_seq(weigth_set, l[i]))

    # Aggiungere i constraint impliciti
    
    # Distance constraint ############################################à
    distance_set = [[] for _ in range(m)] 
    f = Bool('flag')
    for i in range(m): 
        f = True
        for k in range(n-1): 
            for ii in [path[i][j][k] for j in range(n)]: 
                Implies(And(ii, f), distance_set[i].extend([ii] * D[n+1][ii]))
                f = False
                for jj in [path[i][j][k+1] for j in range(n)]:
                    Implies(And(ii,jj), distance_set[i].extend([ii] * D[ii][jj]))
                    Implies(Not([path[i][j][k+1] for j in range(n)]), distance_set[i].extend([ii] * D[ii][n+1]))
          
    while np.sum(flattened_distances[:max_k]) <= upperBound and max_k < n:
        max_k += 1

    # Gestire l'ottimizzazione
    while True: 
        if s.check() == sat:
            m = s.model()
            return [(i, j, k) for i in range(m) for j in range(n) for k in range(n) if m.evaluate(path[i][j][k])]
        else:
            print("unsat")



### Altro metodo

In [ ]:

def MCP(instance, timeout = 5000):

    m = instance['m']
    n = instance['n']
    l = instance['l']
    s = instance['s']
    D = instance['D']

    # c courier è andato da j a k
    # i righe oggetto
    # j colonne oggetto
    path = [[[Bool(f"p_{c}_{i}_{j}") for j in range(n+1)] for i in range(n+1)] for c in range(m)]

    # considerando
    max_cost = ceil(n/m)*max_d
    
    s = Solver()
    s.set(timeout=timeout) # priority='pareto

    # tutti partono da n+1 e finiscono a n+1
    for c in range(m):
        s.add(exactly_one_seq([path[c][n+1][j] for j in range(n+1)]))
        s.add(exactly_one_seq([path[c][i][n+1] for i in range(n+1)]))
    
    # ogni items viene preso da massimo un courier  
    for i in range(n+1):
        for j in range(n+1):
            s.add(at_most_one_seq([path[c][i][j] for c in range(m)]))
    
    # catena di couriers
    for c in range(m):
        for i in range(n+1):
            for j in range(n+1):
                s.add(Implies(path[c][i][j], exactly_one_seq([path[c][j][k] for k in range(n+1)])))

    for c in range(m):
        for j in range(n+1): 
            s.add(at_most_one_seq([path[c][i][j] for i in range(n+1)]))
    
    for c in range(m):
        for i in range(n+1): 
            s.add(at_most_one_seq([path[c][i][j] for j in range(n+1)]))

    # Maximum load constraint 
    for c in range(m):
        weigth_set = []
        for i in range(n):
            for k in range(n+1):
                weigth_set.extend([path[c][i][j]] * s[i])
        s.add(at_most_k_seq(weigth_set, l[c]))

    # Aggiungere i constraint impliciti------------------------
    
    # Distance constraint ############################################à
    distance_set = [[] for _ in range(m)] 
    for c in range(m): 
        for i in range(n+1): 
            for j in range(n+1):
                distance_set[c].extend([path[c][i][j]] * D[i])
                
          
    # capire come fare l'ottimizzazione -> git hub

    # Gestire l'ottimizzazione
    while True: 
        if s.check() == sat:
            m = s.model()
            return [(i, j, k) for i in range(m) for j in range(n) for k in range(n) if m.evaluate(path[i][j][k])]
        else:
            print("unsat")



### SAT con adder 